# Part 3: Semantic Search & Retrieval Augmented Generation (RAG)

## Finde die passende Pizza mit Semantic Search

Task: Ein Kunde soll basierend auf der Beschreibung einer Speise eine passende Bestelloption finden können. 

## Setup

In [ ]:
# Install required packages
%pip install openai matplotlib scikit-learn umap-learn plotly faiss-cpu numpy tabulate pandas

In [ ]:
# Import necessary libraries
from openai import OpenAI
import json
import faiss
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
import io

# Download Menu 
import urllib.request
import os.path
MENU_URL = "https://raw.githubusercontent.com/jank-bcxp/bcxp_weekend2025_HandsOn_AI/refs/heads/main/menu.py"
urllib.request.urlretrieve(MENU_URL, os.path.basename(MENU_URL))
# Download helpers
HELPERS_URL = "https://raw.githubusercontent.com/jank-bcxp/bcxp_weekend2025_HandsOn_AI/refs/heads/main/hack_helpers.py"
urllib.request.urlretrieve(HELPERS_URL, os.path.basename(HELPERS_URL))

from menu import MENU
from hack_helpers import plot_umap

# Initialize OpenAI client
from google.colab import userdata
openai_client = OpenAI(api_key= userdata.get('openai_api_key'))

## Embeddings & Basic Similarity Search

In der Einführung zu LLMs haben wir gesehen, wie Begriffe in einen Vektorraum eingebettet (embedded) und numerisch dargestellt werden können. Der Vektor eines Wortes - das **Embedding** - repräsentiert dabei dessen Bedeutung. 
Bei einer **syntaktischen** Suche nach dem Begriff "Unterkunft" in einem Dokument, werden genau diejenigen Wörter gesucht, die Zeichen für Zeichen mit "Unterkunft" übereinstimmen. Bei der **semantischen** Suche werden hingegen alle Wörter gesucht, die eine ähnliche Bedeutung wie "Unterkunft" besitzen. Das sind diejenigen Wörter, die im Vektorraum 'nahe' am Vektor von "Unterkunft" liegen. Neben Unterkunft würden so auch Begriffe wie "Ferienwohnung" oder "Hotel" auftauchen. Nachfolgend werden wir sehen, wie eine semantische Suche implementiert werden kann. Da der Vektorraum ein kontinuierlicher Raum ist werden bei der semantischen Suche entweder die top-k Ergebnisse verwendet die am nächsten sind oder es wird ein fester Schwellenwert verwendet, der definiert ob ein Kandidat nah genug am gesuchten Begriff ist um als 'Match' zu gelten.

Um Embeddings zu erstellen können unterschiedliche Sprachmodelle verwendet werden. Beispielsweise sind _SentenceTransformer_ eine ressourcen-schonende und frei verfügbare Alternative zu kostenpflichtigen Modellen wie von OpenAI, die auch lokal laufen können. Der einfachheit halber verwenden wir hier aber die OpenAI API um Embeddings zu erzeugen.

UMAP (Uniform Manifold Approximation and Projection) ist ein Verfahren zur Dimensionsreduktion, das hochdimensionale Vektoren – z. B. Embeddings aus Sprache – in einen niederdimensionalen Raum wie 2D für Darstellungszwecke abbildet. Es versucht dabei, semantische Nähe (z. B. zwischen Texten) möglichst gut zu bewahren, indem es lokale Nachbarschaften erhält.

⚠️ Wichtig: Die sichtbare Distanz im 2D-Plot entspricht nicht exakt der tatsächlichen Ähnlichkeit im ursprünglichen Vektorraum. Die Projektion ist eine visuelle Annäherung, keine metrische 1:1-Abbildung.

Führe die nächste Zelle aus um für jeden Menü Eintrag die Nähe zur User-Eingabe zu bestimmen und um die Embeddings der Speisen aus der Speisekarte und den User Input mittels UMAP in einem Plot zu visualisieren.

**Aufgabe:** Füge der Karte einige Getränke und Salate hinzu und beobachte wie sich diese in die UMAP Projektion einfügen. Füge unpassende Nutzer-Eingaben ein und beobacht den Score und wie sich die Anfrage in die UMAP Projektion einfügt.

In [ ]:
# 🗣 Benutzereingabe
user_input = "Ich habe Lust auf eine spicy Pizza."

# 🧾 Strings zur Embedding-Vorbereitung
menu_texts = [f"{item['name']} – {item['beschreibung']}" for item in MENU]
texts_to_embed = [user_input] + menu_texts  # User prompt + all items

# 🧠 Embedding-Berechnung mit OpenAI
response = openai_client.embeddings.create(
    model="text-embedding-3-small", input=texts_to_embed
)
embeddings = [np.array(d.embedding) for d in response.data]

# 🧮 Vektorvergleich (cosine similarity)
user_embedding = embeddings[0] # User Prompt embedding
menu_embeddings = embeddings[1:] # Menu embeddings
# Hier wird die Cosine Similarity zwischen dem User-Embedding und den Menü-Embeddings berechnet
# Für viele Menü-Items und wechselnden Anfragen kann dies sehr ineffizient sein
similarities = cosine_similarity([user_embedding], menu_embeddings)[0] 

# 🥇 Sortierte Ergebnisse
results = []
for idx, score in sorted(enumerate(similarities), key=lambda x: x[1], reverse=True):
    item = MENU[idx]
    results.append(
        {
            "name": item["name"],
            "score": round(score, 3),
            "description": item["beschreibung"],
        }
    )

# 📊 Ausgabe als DataFrame
df = pd.DataFrame(results)
print(df)

plot_umap(user_embedding, menu_embeddings, MENU, similarity_scores=similarities)


## FAISS für schnelle Similarity Search bei großen Datenmengen

**FAISS** (Facebook AI Similarity Search) ist eine Bibliothek für schnelle und skalierbare ähnlichkeitssuche auf Vektoren.
Sie ermöglicht eine effiziente Nearest-Neighbor-Suche — nützlich für semantische Suche, Reccomendation Systems, Clustering usw.

Hauptmerkmale:
- Funktioniert mit hochdimensionalen Embeddings (z. B. von SentenceTransformers)
- Ermöglicht Kosinus-Ähnlichkeit (über normalisierte Vektoren und inneres Produkt) für semantische Ähnlichkeit
- Skaliert auf Millionen oder Milliarden von Vektoren
- Indizes können auf der Festplatte gespeichert und geladen werden – Metadaten müssen jedoch separat verwaltet werden

Typischer Workflow:
1.	Wandle deine Objekte (z. B. Texte) mit einem Embedding-Modell in dichte Vektoren um.
2.	Speichere diese Vektoren in einem FAISS-Index (z. B. IndexFlatIP oder IndexHNSWFlat).
3.	Kodierst du eine Anfrage (Query), kannst du die top-k ähnlichsten Objekte suchen.

**Aufgabe 1**: Finde ein Beispiel für eine User Query, für welches das Ergebnis mit dem besten Ähnlichkeitsscore nicht zur Query passt.

In [ ]:
# === EMBEDDING SETUP ===
# Wir wollen Embeddings erzeugen für Name + Beschreibung der Menüpunkte
menu_texts = [f"{item['name']}: {item['beschreibung']}" for item in MENU]

# Generiere die Embeddings für die Menütexte mit OpenAI
response = openai_client.embeddings.create(model="text-embedding-3-small", input=menu_texts)
embeddings = np.array([d.embedding for d in response.data], dtype=np.float32)

# === FAISS VECTOR INDEX ===
# Normalisierung wird benötigt um zusammen mit inner product (dot product) die Cosine Similarity zu berechnen, welche die Ähnlichkeit der Vektoren beschreibt
faiss.normalize_L2(embeddings)

dim = embeddings.shape[1] # Dimension der Vektoren
index = faiss.IndexFlatIP(dim) # Initialisierung des Indexes mit inner product (dot product)
index.add(embeddings) # Vektoren dem Index hinzufügen

# === MENÜ ABFRAGE ===
def retrieve_menu_items(query: str, top_k: int = 10) -> List[Dict]:
    # Step 1: Generiere Embedding für die User Abfrage
    response = openai_client.embeddings.create(
        model="text-embedding-3-small", input=[query]
    )
    query_vec = np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)

    # Step 2: Normalisiere den Vektor für die Cosine Similarity
    faiss.normalize_L2(query_vec)

    # Step 3: Suche im FAISS Index nach den ähnlichsten Vektoren
    distances, indices = index.search(query_vec, top_k)

    # Step 4: Erstelle eine Liste von Menüpunkten mit den entsprechenden Ähnlichkeitswerten
    return list(zip(indices[0], distances[0]))

print("FAISS Index Description")
print("-" * 30)
print(f"Index Type      : {type(index).__name__}")
print(f"Dimension (D)   : {index.d}")
print(f"Vectors Stored  : {index.ntotal}")
print("-" * 30 + "\n")

In [ ]:
# === EXAMPLE USAGE ===
user_query = "Ich hätte gerne etwas mit Pilzen aber ohne Trüffel"
retrieved_results = retrieve_menu_items(user_query)
for idx, score in retrieved_results:
    print(f"- {MENU[idx]['name']} (score: {score}): {MENU[idx]['beschreibung']}")

## Retrieval Augmented Generation (RAG) mit FAISS und OpenAI API

RAG (Retrieval-Augmented Generation) ist ein KI-Ansatz, der generative Sprachmodelle (z. B. GPT) mit einer externen Wissensquelle kombiniert. Dabei werden relevante Informationen zur Benutzeranfrage zuerst aus einer Datenbank oder Dokumentensammlung _retrieved_ (abgerufen), wie in folgendem Beispiel aus unserem FAISS Index, und anschließend von einem genertiven Modell in die Generierung der Antwort einbezogen. 

- Ein RAG Ansatz kann verwendet werden, wenn auf einem großen Datensatz gearbeitet werden muss, welcher zu groß für den Kontext des Sprachmodells ist.
- Außerdem können durch die Reduzierung des LLM Inputs auf relevante Einträge Kosten gesenkt werden (Anzahl der Input-Tokens sinkt).

**Aufgabe:** Überlege dir Vor und Nachteile eines RAG Ansatzes. In welchen Fällen kann der RAG Ansatz zu schlechten Ergebnissen führen? 

In [ ]:
menu_items_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

user_query = "Ich hätte gerne etwas mit Pilzen aber ohne Trüffel."
retrieved_results = retrieve_menu_items(user_query)
print("Vorselektierte Einträge aus dem Menü:\n")
for idx, score in retrieved_results:
    print(f"- {MENU[idx]['name']} (score: {score}): {MENU[idx]['beschreibung']}")
retrieved_menu_items = [MENU[idx] for idx, _ in retrieved_results]

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "system",
            "content": f"Du bist ein digitaler Kellner. Wähle genau eine passende Speise aus den folgenden vorselektierten Speisen aus, welche die Kriterien aus der Kundenbestellung am ehesten erfüllt:\n{json.dumps(retrieved_menu_items, ensure_ascii=False)}\n",
        },
        {
            "role": "user",
            "content": f"{user_query}",
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": menu_items_schema,
            "strict": True,
        }
    },
)
recommendations = json.loads(response.output_text)["items"]

# 📤 Ausgabe anzeigen
print("\nEmpfohlene Speise:")
for item in recommendations:
    print(item)

## Bonus: Local Embedding Generation

Nachfolgend findet sich ein Beispiel mit der Verwendung eines lokalen Sentence-Transformer Modells für das Erstellen der Embeddings der Speisekarte und des User-Inputs

In [ ]:
from hack_helpers import plot_umap
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# 🗣 2. Benutzereingabe
user_input = "Ich habe Lust auf eine spicy Pizza."

# 🧠 3. Modell laden (z. B. multilingual + performant)
model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# 🧮 4. Embeddings berechnen
menu_texts = [f"{item['name']} – {item['beschreibung']}" for item in MENU] # Hier wird die Speisekarte in emeddable Strings umgewandelt
menu_embeddings = model.encode(menu_texts, convert_to_tensor=True) # Hier werden die Embeddings für die gesamte Speisekarte berechnet
user_embedding = model.encode(user_input, convert_to_tensor=True) # Hier wird das Embedding für die Benutzereingabe berechnet

# TODO Print ein Embedding um die Vectorrepräsentation zu sehen
# print(user_embedding)

# 🔎 5. Ähnlichkeit berechnen
cos_scores = util.cos_sim(user_embedding, menu_embeddings)[0] # Hier wird die Ähnlichkeit zwischen der Benutzereingabe und den Speisen mittels cosine similarity berechnet

# 🥇 6. Sorted results
top_results = sorted(list(enumerate(cos_scores)), key=lambda x: x[1], reverse=True)

# 📤 7. Ausgabe strukturieren
results = []
for idx, score in top_results:
    item = MENU[idx]
    results.append(
        {
            "name": item["name"],
            "score": round(score.item(), 3),
            "description": item["beschreibung"],
        }
    )

# 📊 Als DataFrame anzeigen
pd.set_option("display.width", 200)  # Increase pd width to print correctly
df = pd.DataFrame(results)
print(df)
plot_umap(user_embedding, menu_embeddings, MENU, similarity_scores=cos_scores)